In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [4]:
erf = pd.read_csv('https://github.com/QuantEcon/ShenzhenWinterCamp/raw/master/Jiao%20Shi/ER_predict.csv')
erf['lner'] = np.log(erf[['CNYUSD']])
erf['dep'] = 100*erf['lner'].diff(periods = 1)
erf['int_diff'] = erf['CHN_IBR'] - erf['US_FFR']

,Date,CNYUSD,RMB_REER,CHN PMI,USA PMI,CHN M1,USA M1,CHN M2,USA M2,CHN CPI inflation,USA CPI inflation,CHN_IBR,US_FFR
0,2005-07,8.1062,85.01,51.1,52.8,"97,700.00","1,365.00","277,000.00","6,498.40",1.8,3.2,1.1395,3.36
1,2005-08,8.0998,85.05,52.6,52.4,"99,400.00","1,376.50","281,300.00","6,522.50",1.3,3.6,1.1774,3.30
2,2005-09,8.0920,85.29,55.1,56.8,"101,000.00","1,363.10","287,000.00","6,555.00",0.9,4.7,1.1675,3.57
3,2005-10,8.0845,86.39,54.1,57.2,"101,800.00","1,365.10","287,600.00","6,582.30",1.2,4.3,1.2442,3.73
4,2005-11,8.0804,87.75,54.1,56.7,"104,100.00","1,373.20","292,400.00","6,623.00",1.3,3.5,1.2049,3.99


### UIP

In [14]:
erf['dep_1'] = 0
size_sample = 149

for i in range(size_sample):
    erf.loc[i,'dep_1'] = erf.loc[i+1,'dep']
    
erf.loc[size_sample,'dep_1'] = erf.loc[0,'dep']

In [104]:
erf['const'] = 1 # create column of constant
predicted_s = np.zeros(size_sample)
reg_int = sm.OLS(endog=erf['dep_1'], exog=erf[['const', 'int_diff']], missing='drop')
results = reg_int.fit()
for i in range (size_sample):
    predicted_s[i] = results.predict(exog = [1, erf.loc[i,'int_diff']]) 
predicted_s

array([-0.31715312, -0.31192034, -0.32688106, -0.33133347, -0.34733113,
       -0.33532086, -0.34790304, -0.36672822, -0.36815   , -0.3770067 ,
       -0.36470246, -0.3855427 , -0.38449508, -0.34704784, -0.36677098,
       -0.36282101, -0.35651922, -0.35788755, -0.39862732, -0.38544649,
       -0.39597619, -0.38258156, -0.38943388, -0.3719022 , -0.38675603,
       -0.3734683 , -0.36582491, -0.3482184 , -0.28723702, -0.34112555,
       -0.27957226, -0.25846476, -0.23341263, -0.18905427, -0.18181711,
       -0.15586167, -0.17730591, -0.15913283, -0.16142584, -0.13270703,
       -0.08821505, -0.14623528, -0.1537878 , -0.16008424, -0.16682966,
       -0.16377765, -0.16661586, -0.16596377, -0.15006232, -0.13699908,
       -0.15280432, -0.11403686, -0.14396365, -0.1382124 , -0.14486696,
       -0.13315067, -0.12922742, -0.13744807, -0.13463658, -0.06202443,
       -0.08702845, -0.12953208, -0.07300311, -0.11534639, -0.12401602,
       -0.08956734, -0.05589369, -0.03064914, -0.08928405, -0.10

In [113]:
a=np.array(erf['dep_1'])[:-1]*predicted_s
np.sum(np.where(a>0,1,0))/len(a)

0.71140939597315433

### Taylor

In [106]:
erf['CHNy']=erf['CHN PMI']-np.mean(erf['CHN PMI'])
erf['USAy']=erf['USA PMI']-np.mean(erf['USA PMI'])
erf['PI']=erf['CHN CPI inflation']-erf['USA CPI inflation']
erf['Y']=erf['CHNy']-erf['USAy']
erf.head()

,Date,CNYUSD,RMB_REER,CHN PMI,USA PMI,CHN M1,USA M1,CHN M2,USA M2,CHN CPI inflation,...,US_FFR,lner,dep,dep_1,const,int_diff,CHNy,USAy,PI,Y
0,2005-07,8.1062,85.01,51.1,52.8,"97,700.00","1,365.00","277,000.00","6,498.40",1.8,...,3.36,2.092629,NaN,-0.078983,1,-2.2205,-0.785333,0.137333,-1.4,-0.922667
1,2005-08,8.0998,85.05,52.6,52.4,"99,400.00","1,376.50","281,300.00","6,522.50",1.3,...,3.30,2.091839,-0.078983,-0.096345,1,-2.1226,0.714667,-0.262667,-2.3,0.977333
2,2005-09,8.0920,85.29,55.1,56.8,"101,000.00","1,363.10","287,000.00","6,555.00",0.9,...,3.57,2.090876,-0.096345,-0.092727,1,-2.4025,3.214667,4.137333,-3.8,-0.922667
3,2005-10,8.0845,86.39,54.1,57.2,"101,800.00","1,365.10","287,600.00","6,582.30",1.2,...,3.73,2.089949,-0.092727,-0.050727,1,-2.4858,2.214667,4.537333,-3.1,-2.322667
4,2005-11,8.0804,87.75,54.1,56.7,"104,100.00","1,373.20","292,400.00","6,623.00",1.3,...,3.99,2.089441,-0.050727,-0.126311,1,-2.7851,2.214667,4.037333,-2.2,-1.822667


In [111]:
predicted_s_t = np.zeros(size_sample)
reg_int_t = sm.OLS(endog=erf['dep_1'], exog=erf[['const', 'PI','Y']], missing='drop')
results_t = reg_int_t.fit()
for i in range (size_sample):
    predicted_s_t[i] = results_t.predict(exog = [erf.loc[i,['const','PI','Y']]]) 
predicted_s_t

array([-0.01010534,  0.02992546,  0.13770282,  0.10593746,  0.04645748,
        0.00723591,  0.04272996,  0.08616505,  0.04191842,  0.00804514,
        0.05342539,  0.04532157,  0.0856779 ,  0.04872586, -0.11204957,
       -0.14300455, -0.1444637 , -0.15613228, -0.16164334, -0.13830388,
       -0.18465727, -0.18336089, -0.17914621, -0.23019798, -0.31447427,
       -0.41236543, -0.36082766, -0.31188151, -0.30993751, -0.30896551,
       -0.30442645, -0.44656309, -0.45677483, -0.477682  , -0.36212404,
       -0.25726382, -0.14024787, -0.07282685, -0.1434917 , -0.17477107,
       -0.20880482, -0.24348963, -0.25596974, -0.10864872, -0.19373654,
       -0.17428737, -0.19632815, -0.15159552, -0.15613228, -0.1326312 ,
       -0.14008626, -0.08514419, -0.04268224, -0.07136885, -0.03101251,
       -0.1162608 , -0.08627781, -0.12760614, -0.14219187, -0.18611297,
       -0.19729555, -0.20685737, -0.24381056, -0.28286937, -0.33700105,
       -0.2718484 , -0.2639062 , -0.22747248, -0.22896395, -0.19

In [114]:
a_t=np.array(erf['dep_1'])[:-1]*predicted_s_t
np.sum(np.where(a_t>0,1,0))/len(a_t)

0.64429530201342278

### Random Walk

In [123]:
for i in range(size_sample):
    erf.loc[i+1,'CNYUSD_1'] = erf.loc[i,'CNYUSD']    

In [124]:
erf

,Date,CNYUSD,RMB_REER,CHN PMI,USA PMI,CHN M1,USA M1,CHN M2,USA M2,CHN CPI inflation,...,lner,dep,dep_1,const,int_diff,CHNy,USAy,PI,Y,CNYUSD_1
0,2005-07,8.1062,85.01,51.1,52.8,"97,700.00","1,365.00","277,000.00","6,498.40",1.80,...,2.092629,NaN,-0.078983,1,-2.2205,-0.785333,0.137333,-1.40,-0.922667,NaN
1,2005-08,8.0998,85.05,52.6,52.4,"99,400.00","1,376.50","281,300.00","6,522.50",1.30,...,2.091839,-0.078983,-0.096345,1,-2.1226,0.714667,-0.262667,-2.30,0.977333,8.1062
2,2005-09,8.0920,85.29,55.1,56.8,"101,000.00","1,363.10","287,000.00","6,555.00",0.90,...,2.090876,-0.096345,-0.092727,1,-2.4025,3.214667,4.137333,-3.80,-0.922667,8.0998
3,2005-10,8.0845,86.39,54.1,57.2,"101,800.00","1,365.10","287,600.00","6,582.30",1.20,...,2.089949,-0.092727,-0.050727,1,-2.4858,2.214667,4.537333,-3.10,-2.322667,8.0920
4,2005-11,8.0804,87.75,54.1,56.7,"104,100.00","1,373.20","292,400.00","6,623.00",1.30,...,2.089441,-0.050727,-0.126311,1,-2.7851,2.214667,4.037333,-2.20,-1.822667,8.0845
5,2005-12,8.0702,87.59,54.3,55.1,"107,300.00","1,396.90","298,800.00","6,676.30",1.60,...,2.088178,-0.126311,-0.116546,1,-2.5604,2.414667,2.437333,-1.80,-0.022667,8.0804
6,2006-01,8.0608,86.88,52.1,55.0,"107,400.00","1,375.20","303,500.00","6,674.40",1.90,...,2.087013,-0.116546,-0.239717,1,-2.7958,0.214667,2.337333,-2.10,-2.122667,8.0702
7,2006-02,8.0415,88.13,52.1,55.8,"104,400.00","1,361.70","304,500.00","6,691.80",0.90,...,2.084616,-0.239717,-0.305135,1,-3.1480,0.214667,3.137333,-2.70,-2.922667,8.0608
8,2006-03,8.0170,86.90,55.3,54.3,"107,000.00","1,394.40","310,500.00","6,764.30",0.80,...,2.081564,-0.305135,-0.006237,1,-3.1746,3.414667,1.637333,-2.60,1.777333,8.0415
9,2006-04,8.0165,85.98,58.1,55.2,"106,400.00","1,393.30","313,700.00","6,827.60",1.20,...,2.081502,-0.006237,0.028687,1,-3.3403,6.214667,2.537333,-2.30,3.677333,8.0170


In [125]:
predicted_s_rw = np.zeros(size_sample)
reg_int_rw = sm.OLS(endog=erf.loc[1:149,'CNYUSD'], exog=erf.loc[1:149,'CNYUSD_1'], missing='drop')
results_rw = reg_int_rw.fit()
results_rw.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 CNYUSD   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 3.550e+06
Date:                Fri, 02 Feb 2018   Prob (F-statistic):               0.00
Time:                        16:50:29   Log-Likelihood:                 254.22
No. Observations:                 149   AIC:                            -506.4
Df Residuals:                     148   BIC:                            -503.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
CNYUSD_1       0.9983      0.001   1884.017      0.000       0.997       0.999
==============================================================================
Omnibus:                       86.104   Durbin-Watson:                   1.399
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              867.891
Skew:                           1.800   Prob(JB):                    3.47e-189
Kurtosis:                      14.262   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

This is a random walk. But beta is less than 1, predicting that the currency keeps depreciating. So 

In [129]:
a_rw=np.array(erf['dep'])
np.sum(np.where(a<0,1,0))/len(a)

0.28187919463087246

### Conclusion

UIP model performs the best. UIP model and the Taylor Rule model both beat the random walk model.